In [17]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%autoreload 2

In [38]:
import numpy as np
from experiments.founder_rank.helpers import *
import experiments.founder_rank.metrics as metric
import experiments.founder_rank.learner as learn

In [39]:
import experiments.founder_rank.data.private.baseline as baseline
import experiments.founder_rank.data.private.random as random
import experiments.founder_rank.data.private.naive as naive
import experiments.founder_rank.data.private.graph_metrics as graph_metrics

In [40]:
datasets = get_datasets([baseline, random, naive])
with_graph_metrics = get_dataset(graph_metrics)

In [41]:
metrics = metric.Metrics(datasets['baseline'])
learner = learn.Learner(datasets['baseline'])

In [42]:
only_pr = with_graph_metrics.take([0, 1], 1)
only_betweenness = with_graph_metrics.take([0, 2], 1)
only_closeness = with_graph_metrics.take([0, 3], 1)
np.corrcoef(with_graph_metrics[:, 1:], rowvar=False)

array([[1.        , 0.91689477, 0.53587024],
       [0.91689477, 1.        , 0.51547319],
       [0.53587024, 0.51547319, 1.        ]])

In [43]:
VERSIONS = {
  'only_pr': [1], 
  'only_betweenness': [2],
  'only_closeness': [3],
  'pr_and_closeness': [1, 3],
  'weighted': [1, 2, 3],
}

for name, indexes in VERSIONS.items():
  features = with_graph_metrics.take([0] + indexes, 1)
  dataset, coeff, r2 = learner.linear_regression(features)
  datasets[name] = dataset
  print('{n}: r^2={r2}, coeff={c}'.format(n=name, r2=r2, c=coeff))

only_pr: r^2=-0.1333728765346942, coeff=[1.4943471]
only_betweenness: r^2=-0.2929714179906835, coeff=[1.5737442]
only_closeness: r^2=0.30673542058554815, coeff=[0.45353152]
pr_and_closeness: r^2=0.35416518931646945, coeff=[0.44676612 0.36929139]
weighted: r^2=0.354549747149452, coeff=[0.37014197 0.09010324 0.37006656]


In [44]:
print(metrics.idcg)
for_each_dataset(datasets, metrics.ndcg)

2.1435899981703272e+188
baseline: 1.0
random: 0.157310319642317
naive: 0.39869350465721554
only_pr: 0.5249807203302599
only_betweenness: 0.3652083093261328
only_closeness: 0.34657434496786554
pr_and_closeness: 0.5272585189085583
weighted: 0.44131327679884746


In [45]:
for n in [5, 10, 20, 50]:
  print('n = {n}'.format(n=n))
  for_each_dataset(datasets, lambda a: metrics.precision_at(n, a))

n = 5


baseline: 1.0
random: 0.0
naive: 0.2
only_pr: 0.2
only_betweenness: 0.2
only_closeness: 0.2
pr_and_closeness: 0.2
weighted: 0.2
n = 10
baseline: 1.0
random: 0.0
naive: 0.2
only_pr: 0.1
only_betweenness: 0.2
only_closeness: 0.2
pr_and_closeness: 0.2
weighted: 0.2
n = 20
baseline: 1.0
random: 0.1
naive: 0.2
only_pr: 0.15
only_betweenness: 0.2
only_closeness: 0.25
pr_and_closeness: 0.2
weighted: 0.2
n = 50
baseline: 1.0
random: 0.04
naive: 0.34
only_pr: 0.34
only_betweenness: 0.3
only_closeness: 0.24
pr_and_closeness: 0.34
weighted: 0.36


In [46]:
for_each_dataset(datasets, metrics.tau)

baseline: KendalltauResult(correlation=1.0, pvalue=4.663665584756559e-306)
random: KendalltauResult(correlation=-0.030410256410256412, pvalue=0.2554547652516961)
naive: KendalltauResult(correlation=0.5827897435897437, pvalue=2.6693803771270578e-105)
only_pr: KendalltauResult(correlation=0.41196923076923087, pvalue=1.4985409960734286e-53)
only_betweenness: KendalltauResult(correlation=0.4338564102564103, pvalue=3.402392102766875e-59)
only_closeness: KendalltauResult(correlation=0.48533333333333345, pvalue=1.3035081373469383e-73)
pr_and_closeness: KendalltauResult(correlation=0.48145641025641034, pvalue=1.8061148181007626e-72)
weighted: KendalltauResult(correlation=0.48213333333333336, pvalue=1.1430589980383511e-72)


In [47]:
for_each_dataset(datasets, metrics.rho)

baseline: SpearmanrResult(correlation=1.0, pvalue=0.0)
random: SpearmanrResult(correlation=-0.046381420496436475, pvalue=0.24693036736928112)
naive: SpearmanrResult(correlation=0.7604095846645368, pvalue=6.860797865181096e-119)
only_pr: SpearmanrResult(correlation=0.5774893094126322, pvalue=7.187626389684997e-57)
only_betweenness: SpearmanrResult(correlation=0.6084856230031949, pvalue=1.4419932528515797e-64)
only_closeness: SpearmanrResult(correlation=0.6689967068075695, pvalue=2.5170520604099495e-82)
pr_and_closeness: SpearmanrResult(correlation=0.6613519292209388, pvalue=7.476381815859652e-80)
weighted: SpearmanrResult(correlation=0.6623923814204965, pvalue=3.478820824851354e-80)


In [48]:
for_each_dataset(datasets, metrics.rmse)

baseline: 0.0
random: 0.24814771023204962
naive: 0.07876901961651882
only_pr: 0.14266688304365666
only_betweenness: 0.18077572063367597
only_closeness: 0.13347453609204107
pr_and_closeness: 0.10002429059125684
weighted: 0.09947651694583434


In [49]:
for_each_dataset(datasets, metrics.mae)

baseline: 0.0
random: 0.22936083268856614
naive: 0.05888074292785914
only_pr: 0.11407834442021708
only_betweenness: 0.14680929114886246
only_closeness: 0.09106674051023646
pr_and_closeness: 0.07296182375927146
weighted: 0.07293279368735266
